# Load and inspect the Dataset

In [31]:
import pandas as pd

# Data URL
DATA_URL = "https://raw.githubusercontent.com/Sven-Bo/datasets/master/store_sales_2022-2023.csv"

# Load Dataset to pandas
data = pd.read_csv(DATA_URL)

# Display the data
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount
0,1,52,1,CodeComet,Software Development Tools,Tokyo,1/1/2022,8,303.29
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,1/1/2022,8,173.53
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,1/2/2022,6,37.72
3,4,88,2,VarVista Pro,Educational Tools,Osaka,1/2/2022,6,10.47
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,1/3/2022,1,159.10


## Check data type

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          1730 non-null   int64  
 1   product_id        1730 non-null   int64  
 2   store_id          1730 non-null   int64  
 3   product_name      1730 non-null   object 
 4   product_category  1730 non-null   object 
 5   city              1730 non-null   object 
 6   date_of_sale      1730 non-null   object 
 7   quantity_sold     1730 non-null   int64  
 8   sales_amount      1730 non-null   float64
dtypes: float64(1), int64(4), object(4)
memory usage: 121.8+ KB


# Transform Data for Datetime and Extract features

In [33]:
# Convert 'data_of_sale' to datetime
data['date_of_sale'] = pd.to_datetime(data['date_of_sale'])

# Check data type
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          1730 non-null   int64         
 1   product_id        1730 non-null   int64         
 2   store_id          1730 non-null   int64         
 3   product_name      1730 non-null   object        
 4   product_category  1730 non-null   object        
 5   city              1730 non-null   object        
 6   date_of_sale      1730 non-null   datetime64[ns]
 7   quantity_sold     1730 non-null   int64         
 8   sales_amount      1730 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 121.8+ KB


## Extract month and year

In [34]:
data['month'] = data['date_of_sale'].dt.month
data['year'] = data['date_of_sale'].dt.year

# Check the extracted data
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount,month,year
0,1,52,1,CodeComet,Software Development Tools,Tokyo,2022-01-01,8,303.29,1,2022
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,2022-01-01,8,173.53,1,2022
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,2022-01-02,6,37.72,1,2022
3,4,88,2,VarVista Pro,Educational Tools,Osaka,2022-01-02,6,10.47,1,2022
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,2022-01-03,1,159.10,1,2022


# Chaining Transformations

In [39]:
# Perform the same transformation using chaining method
data = data.assign(
    date_of_sale=lambda df: pd.to_datetime(df['date_of_sale']),
    month=lambda df: df['date_of_sale'].dt.month,
    year=lambda df: df['date_of_sale'].dt.year
)

# Check the transformed DataFrame
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount,month,year
0,1,52,1,CodeComet,Software Development Tools,Tokyo,2022-01-01,8,303.29,1,2022
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,2022-01-01,8,173.53,1,2022
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,2022-01-02,6,37.72,1,2022
3,4,88,2,VarVista Pro,Educational Tools,Osaka,2022-01-02,6,10.47,1,2022
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,2022-01-03,1,159.10,1,2022


# Calculating City Revenues

In [44]:
YEAR = 2023

# Calculate total revenue for each city and year then calculate percentage change
city_revenue = (
    data.groupby(['city', 'year'])['sales_amount']
    .sum()
    .unstack()
    .assign(change=lambda x: x.pct_change(axis=1)[YEAR] *100)
)

city_revenue

year,2022,2023,change
city,,,
Osaka,76914.92,81202.93,5.575004
Tokyo,79961.13,72717.66,-9.058739
Yokohama,63216.90,67110.89,6.159729


## Sales from Tokyo

In [53]:
city = "Tokyo"

revenue = city_revenue.loc[city, YEAR]
change = city_revenue.loc[city, "change"]

print(f'Revenue for {city} is ${revenue:.2f} which is {change:.2f}%')

Revenue for Tokyo is $72717.66 which is -9.06%


# Prepare data for visualization

In [54]:
# Variables for filtering
selected_city = "Tokyo"
visualization_year = 2023

# Filter, group and sum data for bar chart
filtered_data = (
    data.query("city == @selected_city & year == @visualization_year")
    .groupby("month", dropna=False, as_index=False)["sales_amount"]
    .sum()
)

filtered_data

,month,sales_amount
0,1,7289.70
1,2,6281.59
2,3,6568.99
3,4,6307.56
4,5,5881.92
5,6,4662.89
6,7,6116.09
7,8,5553.45
8,9,5327.64
9,10,8650.56
